In [1]:
import pandas as pd
import numpy as np
import random

    Importing Data

In [2]:
df = pd.read_csv("allele.csv",index_col=0)
allele = df["Allele"].to_list()

df = pd.read_csv("rooms.csv",index_col=0)
rooms = df["Rooms"].to_list()

df = pd.read_csv("slots.csv",index_col=0)
slots = df["Slots"].to_list()

In [3]:
for i in range(0,len(allele)):
    allele[i] = eval(allele[i])

In [4]:
all_courses = []
for a,b,c,d in allele:
    if a not in all_courses:
        all_courses.append(a)

    Genetic Algorithm

In [5]:
#Initial TimeTable Creation 
def makeTimeTable(populationSize,population):
    for i in range(0,populationSize):
        exist = []
        timetable = np.zeros((len(rooms), len(slots)), dtype=tuple)

        for i in range(0,len(timetable)):
            for j in range(0,len(timetable[i])):

                while True:
                    pos = random.randint(0, len(allele)-1)
                    if pos not in exist:
                        break

                timetable[i][j] = allele[pos]
                exist.append(pos)

        population.append(timetable)

In [6]:
population = []
makeTimeTable(50,population)

#Fitness Function which starts from 0 and as invalid constraints are found the score will be increased
#Meaning the timetable with 0 score will be fulfilling every condition
def fitness(candidate):
    score = 0

    #For Hard Constraints
    track_teacher = 0
    track_section = 0

    for i in range(0,len(slots)):   #Column
        teacher_occur = []
        section_occur = []

        for j in range(0,len(rooms)): #Row

            course, section, teacher, batch = candidate[j][i]

            if teacher in teacher_occur:
                track_teacher+=1
            else:
                teacher_occur.append(teacher)

            if section in section_occur:
                track_section+=1
            else:
                section_occur.append(section)

        del teacher_occur
        del section_occur


    score = track_section + track_teacher

    #Soft Constraints

    count_section = {}
    count_teacher = {}

    called_section = 0
    called_teacher = 0

    for i in range(0,len(slots)):   #Column
        for j in range(0,len(rooms)): #Row

            course, section, teacher, batch = candidate[j][i]

            if teacher in count_teacher:
                count_teacher[teacher] += count_teacher[teacher] + 1
            else:
                count_teacher[teacher] = 0

            if section in count_section:
                count_section[teacher] += count_section[teacher] + 1
            else:
                count_section[teacher] = 0
    

    for key in count_section:
        if count_section[key] > 3:
            called_section += 1

    for key in count_teacher:
        if count_teacher[key] > 3:
            called_teacher += 1


    score += called_section
    score += called_teacher


    del track_section
    del track_teacher
    del count_section
    del count_teacher
    del called_section
    del called_teacher

    return score

In [7]:
#Tournament Selection is used with sample size as 5 and best 2 will be chosen at each iteration
def selection(population):
    
    tour_size = 5
    tour_sample = random.sample(population, tour_size)
    tour_fitness = []

    for i in tour_sample:
        x = fitness(i)
        tour_fitness.append((i,x))
    
    tour_fitness = sorted(tour_fitness, key = lambda a: a[1])

    parents = []
    for i in range(0,2):
        parents.append(tour_fitness[i][0])

    del tour_fitness
    del tour_sample

    return parents

In [8]:
# Interchange both parents from random point(crossover_point) and create two children
def crossover(parents):

    parent1 = parents[0]
    parent2 = parents[1]

    child1 = []
    child2 = []

    crossover_point = random.randint(1, len(parent1)-1)
    
    child1 = np.vstack((parent1[:crossover_point,:],parent2[crossover_point:,:]))
    child2 = np.vstack((parent2[:crossover_point,:],parent1[crossover_point:,:]))

    return [child1, child2]

In [9]:
#At every 5 iterations mutations will happen within the population
def mutation(individual,iteration):
    
    if iteration%5 == 0:
        
        row = random.randint(0, len(individual) - 1)
        col = random.randint(0, len(individual[0]) - 1)
        
        old_val = individual[row][col]
        new_val = random.choice([x for x in allele if x != old_val])
        
        individual[row][col] = new_val

    return individual

In [10]:
generation = 0

#Runner Code for GEnetic Algorithm
while True:

    new_population = []
    for i in range(len(population)//2):
        parents = selection(population)
        
        child1, child2 = crossover(parents)
        
        child1 = mutation(child1,generation)
        child2 = mutation(child2,generation)

        new_population.append(child1)
        new_population.append(child2)

    all_fit = []
    for i in range(len(population)):
        temp = fitness(population[i])
        all_fit.append((population[i],temp))
    
    all_fit = sorted(all_fit, key = lambda a: a[1])

    for j in range(0,len(population)):
        population[j] = new_population[j]

    del new_population


    store_fit = []    

    # Print the best individual
    for i in range(0,len(population)):
        store_fit.append((population[i],fitness(population[i])))

    store_fit = sorted(store_fit, key = lambda a: a[1])
    best_individual = store_fit[0][0]

    print("Generation:",generation+1,"Fitness:",fitness(best_individual))


    if store_fit[0][1] == 0:
        
        print(f"Generation {generation+1}")
        print("Solution found!")
        break

    del store_fit

    generation+=1

Generation: 1 Fitness: 43
Generation: 2 Fitness: 41
Generation: 3 Fitness: 45
Generation: 4 Fitness: 44
Generation: 5 Fitness: 42
Generation: 6 Fitness: 43
Generation: 7 Fitness: 39
Generation: 8 Fitness: 37
Generation: 9 Fitness: 39
Generation: 10 Fitness: 42
Generation: 11 Fitness: 39
Generation: 12 Fitness: 35
Generation: 13 Fitness: 34
Generation: 14 Fitness: 37
Generation: 15 Fitness: 35
Generation: 16 Fitness: 36
Generation: 17 Fitness: 35
Generation: 18 Fitness: 35
Generation: 19 Fitness: 34
Generation: 20 Fitness: 34
Generation: 21 Fitness: 33
Generation: 22 Fitness: 32
Generation: 23 Fitness: 30
Generation: 24 Fitness: 30
Generation: 25 Fitness: 30
Generation: 26 Fitness: 29
Generation: 27 Fitness: 29
Generation: 28 Fitness: 28
Generation: 29 Fitness: 27
Generation: 30 Fitness: 28
Generation: 31 Fitness: 27
Generation: 32 Fitness: 27
Generation: 33 Fitness: 26
Generation: 34 Fitness: 26
Generation: 35 Fitness: 26
Generation: 36 Fitness: 25
Generation: 37 Fitness: 25
Generation

    Displaying TimeTable with Teacher Details

In [11]:
all_columns = []
lst = [list(row) for row in best_individual]

for i in range(0,6):
    all_columns.append([row[i] for row in lst])

df = pd.DataFrame()
df["Room\\Time"] = rooms

for i in range(0,6):
    df[slots[i]] = all_columns[i]


df.set_index("Room\\Time",inplace=True)
df


,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
Room\Time,,,,,,
C-301,"(FSPM, CS-8A, Dr. Noreen Jamil, BS (CS/DS/CY) ...","(DAA, CS-4A/Y, Mr. Owais Idrees, BS Batch 2021)","(Technopreneurship, CS-8A, Usman Ahmed, BS Bat...","(PS, CS-2D, Muhammad Ajmal, BS Batch 2022)","(AI, DS-6N, Mr. Umair Arshad, BS Batch 2020)","(MM, CS-4A, Sadaf Zahra Usman, BS Batch 2021)"
C-302,"(PF, F, Mr. Majid Hussain, BS Batch 2022 and E...","(ACV, PCS-2A, Dr. Ahmad Raza Shahid, PhD)","(FM, CS-4B, Ayesha Yaqoob, BS Batch 2021)","(ISE, AI-2C, Muhammad Ibrahim, BS Batch 2022)","(SM, CS-6E, Mr. Muhammad Almas Khan, BS Batch ...","(MAD, MCNS-A, Mr. Jawad Hassan, MS Elective Co..."
C-303,"(OOP, CS-2G, Mr. Shehreyar Rashid, BS Batch 2022)","(OOP, CS-2D, Ms. Amna Irum, BS Batch 2022)","(DS, CY-2A, Ms. Sobia Rasheed, BS Batch 2022)","(ISE, SE-2C, Sobia Shehzadi , BS Batch 2022)","(APP, PCS-3A, Dr. Muhammad Aleem, PhD)","(DS, A, Ms. Madiha Umar, BS Batch 2021 and Ear..."
C-304,"(PS, CS-4D, Moomal Bukhari, BS Batch 2021)","(DBS, AI-4K, Ms. Ayesha Kamran, BS Batch 2021)","(PMS, SE-6R, Dr. Naveed Ahmad, BS Batch 2020)","(CAG, CS-F, Imran Shahzad, BS Batch 2022 and E...","(PMS, SE-6R, Dr. Naveed Ahmad, BS Batch 2020)","(DIP, CS-8B, Dr. Akhtar Jamil, BS (CS/DS/CY) B..."
C-305,"(MM, SE-4P, Bilal Saeed, BS Batch 2021)","(AP, MCNS-A, M. Adil ur Rehman, MS (CNS))","(PS, CS-4C, Waqas Munir, BS Batch 2021)","(ISE, CY-2D, Mahboobullah, BS Batch 2022)","(PMS, SE-6P, Mr. Irfan Ullah, BS Batch 2020)","(WP, CS-B, Ms. Tajwar Mehmood, BS (CS/DS/CY) B..."
C-307,"(CPS, SE-2C, Aniqa Jahanghir, BS Batch 2022)","(OOP, CS-Z, Dr. Javeria Imtiaz, BS Batch 2022)","(DE, CS-2F, Usman Rashid, BS Batch 2022)","(PS, CS-4Y, Usman Ashraf, BS Batch 2021)","(NLP, CS-8A, Dr. Mehreen Alam, BS (CS/DS/CY) B...","(ISE, CY-2C, Mahboobullah, BS Batch 2022)"
C-308,"(DE, AI-2D, Sehrish Hassan, BS Batch 2022)","(CN, CY-4T, Dr. Muhammad Asim, BS Batch 2021)","(CN, AI-6J, Dr. Usman Habib, BS Batch 2020)","(OS, CS-4E/V, Mr. Shoaib Mehboob, BS Batch 2021)","(DAA, CS-4B/Z, Ms. Nirmal Tariq, BS Batch 2021)","(TBW, CY-4T, Amna Hafeez, BS Batch 2021)"
C-309,"(CC, CS-8B, Dr. Mudassar Aslam, BS (CS/DS/CY) ...","(CPS, CS-2C, Zunera Malik, BS Batch 2022)","(PS, CS-4H, Sana Saleh (EE), BS Batch 2021)","(GL, SE-4Q, Ebrahim Riaz, BS Batch 2021)","(DE, DS-2A, Sumaira Azhar, BS Batch 2022)","(DLD, CS-2H, Mr. M. Awais (EE), BS Batch 2022)"
C-310,"(RM, MCNS-A, Dr. Hammad Majeed, MS (CNS))","(ISE, SE-2A, Ahmed Abdul Rahman, BS Batch 2022)","(OOP, CS-2G, Mr. Shehreyar Rashid, BS Batch 2022)","(OOP, SE-2F, Ms. Uzma Mahar, BS Batch 2022)","(PS, CS-2A, Farooq Dar, BS Batch 2022)","(DE, CY-2D, Javaria Akram, BS Batch 2022)"


    Displaying TimeTable with Course and Sections

In [12]:
cleaned_list = []

for j in range(6):
    cleaned_list.append([all_columns[j][i][0] + " " +all_columns[j][i][1] for i in range(len(all_columns[i]))])

clean_df = pd.DataFrame()
clean_df["Room\\Time"] = rooms

for i in range(6):
    clean_df[slots[i]] = cleaned_list[i]

clean_df.set_index("Room\\Time",inplace=True)
clean_df.to_csv("TimeTable.csv")

clean_df

,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
Room\Time,,,,,,
C-301,FSPM CS-8A,DAA CS-4A/Y,Technopreneurship CS-8A,PS CS-2D,AI DS-6N,MM CS-4A
C-302,PF F,ACV PCS-2A,FM CS-4B,ISE AI-2C,SM CS-6E,MAD MCNS-A
C-303,OOP CS-2G,OOP CS-2D,DS CY-2A,ISE SE-2C,APP PCS-3A,DS A
C-304,PS CS-4D,DBS AI-4K,PMS SE-6R,CAG CS-F,PMS SE-6R,DIP CS-8B
C-305,MM SE-4P,AP MCNS-A,PS CS-4C,ISE CY-2D,PMS SE-6P,WP CS-B
C-307,CPS SE-2C,OOP CS-Z,DE CS-2F,PS CS-4Y,NLP CS-8A,ISE CY-2C
C-308,DE AI-2D,CN CY-4T,CN AI-6J,OS CS-4E/V,DAA CS-4B/Z,TBW CY-4T
C-309,CC CS-8B,CPS CS-2C,PS CS-4H,GL SE-4Q,DE DS-2A,DLD CS-2H
C-310,RM MCNS-A,ISE SE-2A,OOP CS-2G,OOP SE-2F,PS CS-2A,DE CY-2D
